In [8]:
import numpy as np
import pandas as pd
import json
from rauth import OAuth1Service,OAuth1Session
import requests 
import webbrowser
import datetime
from requests_oauthlib import OAuth1
import time
import os
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

batch_size = 32
img_height = 180
img_width = 180

class_names = ['rsi_gain', 'rsi_loss']

consumer_key='85bdd7b968b11348fd8682f33a1d1c82'
consumer_secret='1fd4678dd9db8eefeb9046bfcfda129d'

is_long = False
is_long_2 = False
is_short = False
spy_data_dict = {'times':[],'prices':[]}

etrade = OAuth1Service(
    name="etrade",
    consumer_key='85bdd7b968b11348fd8682f33a1d1c82',
    consumer_secret='1fd4678dd9db8eefeb9046bfcfda129d',
    request_token_url="https://api.etrade.com/oauth/request_token",
    access_token_url="https://api.etrade.com/oauth/access_token",
    authorize_url="https://us.etrade.com/e/t/etws/authorize?key={}&token={}",
    base_url="https://api.etrade.com")

request_token, request_token_secret = etrade.get_request_token(
        params={"oauth_callback": "oob", "format": "json"})

authorize_url = etrade.authorize_url.format(etrade.consumer_key, request_token)
webbrowser.open(authorize_url)
text_code = input("Please accept agreement and enter text code from browser: ")
session = etrade.get_auth_session(request_token,
                                  request_token_secret,
                                  params={"oauth_verifier": text_code})
exp_day = str(input('Enter Expiration Day'))
exp_month = str(input('Enter Expiration Month'))

accounts_list_url = "https://api.etrade.com/v1/accounts/list.json"
r = session.get(accounts_list_url)
# r = requests.get(accounts_list_url,auth=auth)
r = json.loads(r.text)
rollover_key = r['AccountListResponse']['Accounts']['Account'][2]['accountIdKey']
def get_data():
    
    r = session.get('https://api.etrade.com/v1/market/quote/SPY.json')
    r = json.loads(r.content)
    spy_market_price = round((r['QuoteResponse']['QuoteData'][0]['All']['ask']+r['QuoteResponse']['QuoteData'][0]['All']['bid'])/2,2)
    curr_time = int(str(datetime.datetime.now().month)+str(datetime.datetime.now().day)+str(datetime.datetime.now().hour)+str(datetime.datetime.now().minute)+str(datetime.datetime.now().second))
    return spy_market_price,curr_time

def buy_open_order():
    preview_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/preview.json'
    client_order_id = str(np.random.randint(1000000000, 2000000000))
    headers = {"Content-Type": "application/xml", "consumerKey": consumer_key}
    payload = """<PreviewOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>MARKET</priceType>
                           <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice></limitPrice>
                           <Instrument>
                               <Product>
                                   <callPut>CALL</callPut>
                                   <expiryDay>{}</expiryDay>
                                   <expiryMonth>{}</expiryMonth>
                                   <expiryYear>2021</expiryYear>
                                   <securityType>OPTN</securityType>
                                   <strikePrice>{}</strikePrice>
                                   <symbol>SPY</symbol>
                               </Product>
                               <orderAction>BUY_OPEN</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>1</quantity>
                           </Instrument>
                       </Order>
                   </PreviewOrderRequest>"""

    payload = payload.format(client_order_id,exp_day,exp_month,strike_price)
    r = session.post(preview_order_url,header_auth=True,headers=headers,data=payload)
    if r.status_code == 200 :
        r = json.loads(r.content)
        if r['PreviewOrderResponse']['totalOrderValue'] < r['PreviewOrderResponse']['cashBpDetails']['settled']['currentBp']:
            preview_id = r['PreviewOrderResponse']['PreviewIds'][0]['previewId']
            place_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/place.json'
            payload = """<PlaceOrderRequest>
                           <orderType>OPTN</orderType>
                           <clientOrderId>{}</clientOrderId>
                           <PreviewIds>
                              <previewId>{}</previewId>
                           </PreviewIds>
                           <Order>
                              <allOrNone>false</allOrNone>
                                  <priceType>MARKET</priceType>
                                  <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                                  <marketSession>REGULAR</marketSession>
                                  <stopPrice></stopPrice>
                                  <limitPrice></limitPrice>
                              <Instrument>
                                 <Product>
                                     <callPut>CALL</callPut>
                                     <expiryDay>{}</expiryDay>
                                     <expiryMonth>{}</expiryMonth>
                                     <expiryYear>2021</expiryYear>
                                     <securityType>OPTN</securityType>
                                     <strikePrice>{}</strikePrice>
                                     <symbol>SPY</symbol>
                                 </Product>
                                 <orderAction>BUY_OPEN</orderAction>
                                 <quantityType>QUANTITY</quantityType>
                                 <quantity>1</quantity>
                              </Instrument>
                           </Order>
                        </PlaceOrderRequest>"""

            payload = payload.format(client_order_id,preview_id,exp_day,exp_month,strike_price)
            r = session.post(place_order_url,header_auth=True,headers=headers,data=payload)
            if r.status_code == 200:
                print('SPY Long Executed!')
        else:
            print('SPY Long Place order failed')
    else:
        print('SPY Long Preview order failed')

def sell_close_order(prev_strike_price):
    preview_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/preview.json'
    client_order_id = str(np.random.randint(1000000000, 2000000000))
    headers = {"Content-Type": "application/xml", "consumerKey": consumer_key}
    payload = """<PreviewOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>MARKET</priceType>
                           <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice></limitPrice>
                           <Instrument>
                               <Product>
                                   <callPut>CALL</callPut>
                                   <expiryDay>{}</expiryDay>
                                   <expiryMonth>{}</expiryMonth>
                                   <expiryYear>2021</expiryYear>
                                   <securityType>OPTN</securityType>
                                   <strikePrice>{}</strikePrice>
                                   <symbol>SPY</symbol>
                               </Product>
                               <orderAction>SELL_CLOSE</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>1</quantity>
                           </Instrument>
                       </Order>
                   </PreviewOrderRequest>"""

    payload = payload.format(client_order_id,exp_day,exp_month,prev_strike_price)
    r = session.post(preview_order_url,header_auth=True,headers=headers,data=payload)
    if r.status_code == 200:
        r = json.loads(r.content)
        preview_id = r['PreviewOrderResponse']['PreviewIds'][0]['previewId']
        place_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/place.json'
        payload = """<PlaceOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <PreviewIds>
                          <previewId>{}</previewId>
                       </PreviewIds>
                       <Order>
                          <allOrNone>false</allOrNone>
                              <priceType>MARKET</priceType>
                              <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                              <marketSession>REGULAR</marketSession>
                              <stopPrice></stopPrice>
                              <limitPrice></limitPrice>
                          <Instrument>
                             <Product>
                                 <callPut>CALL</callPut>
                                 <expiryDay>{}</expiryDay>
                                 <expiryMonth>{}</expiryMonth>
                                 <expiryYear>2021</expiryYear>
                                 <securityType>OPTN</securityType>
                                 <strikePrice>{}</strikePrice>
                                 <symbol>SPY</symbol>
                             </Product>
                             <orderAction>SELL_CLOSE</orderAction>
                             <quantityType>QUANTITY</quantityType>
                             <quantity>1</quantity>
                          </Instrument>
                       </Order>
                    </PlaceOrderRequest>"""

        payload = payload.format(client_order_id,preview_id,exp_day,exp_month,prev_strike_price)
        r = session.post(place_order_url,header_auth=True,headers=headers,data=payload)
        if r.status_code == 200:
            print('SPY Long Closed!')
        else:
            print('SPY Close Place order failed - trying again')
            sell_close_order(prev_strike_price)
    else:
        print('SPY Close Preview order failed - trying again')
        sell_close_order(prev_strike_price)
        
def buy_open_order_put():
    preview_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/preview.json'
    client_order_id = str(np.random.randint(1000000000, 2000000000))
    headers = {"Content-Type": "application/xml", "consumerKey": consumer_key}
    payload = """<PreviewOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>MARKET</priceType>
                           <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice></limitPrice>
                           <Instrument>
                               <Product>
                                   <callPut>PUT</callPut>
                                   <expiryDay>{}</expiryDay>
                                   <expiryMonth>{}</expiryMonth>
                                   <expiryYear>2021</expiryYear>
                                   <securityType>OPTN</securityType>
                                   <strikePrice>{}</strikePrice>
                                   <symbol>SPY</symbol>
                               </Product>
                               <orderAction>BUY_OPEN</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>1</quantity>
                           </Instrument>
                       </Order>
                   </PreviewOrderRequest>"""

    payload = payload.format(client_order_id,exp_day,exp_month,put_strike_price)
    r = session.post(preview_order_url,header_auth=True,headers=headers,data=payload)
    if r.status_code == 200 :
        r = json.loads(r.content)
        if r['PreviewOrderResponse']['totalOrderValue'] < r['PreviewOrderResponse']['cashBpDetails']['settled']['currentBp']:
            preview_id = r['PreviewOrderResponse']['PreviewIds'][0]['previewId']
            place_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/place.json'
            payload = """<PlaceOrderRequest>
                           <orderType>OPTN</orderType>
                           <clientOrderId>{}</clientOrderId>
                           <PreviewIds>
                              <previewId>{}</previewId>
                           </PreviewIds>
                           <Order>
                              <allOrNone>false</allOrNone>
                                  <priceType>MARKET</priceType>
                                  <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                                  <marketSession>REGULAR</marketSession>
                                  <stopPrice></stopPrice>
                                  <limitPrice></limitPrice>
                              <Instrument>
                                 <Product>
                                     <callPut>PUT</callPut>
                                     <expiryDay>{}</expiryDay>
                                     <expiryMonth>{}</expiryMonth>
                                     <expiryYear>2021</expiryYear>
                                     <securityType>OPTN</securityType>
                                     <strikePrice>{}</strikePrice>
                                     <symbol>SPY</symbol>
                                 </Product>
                                 <orderAction>BUY_OPEN</orderAction>
                                 <quantityType>QUANTITY</quantityType>
                                 <quantity>1</quantity>
                              </Instrument>
                           </Order>
                        </PlaceOrderRequest>"""

            payload = payload.format(client_order_id,preview_id,exp_day,exp_month,put_strike_price)
            r = session.post(place_order_url,header_auth=True,headers=headers,data=payload)
            if r.status_code == 200:
                print('SPY Short Executed!')
        else:
            print('SPY Short Place order failed')
    else:
        print('SPY Short Preview order failed')

def sell_close_order_put(prev_strike_price):
    preview_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/preview.json'
    client_order_id = str(np.random.randint(1000000000, 2000000000))
    headers = {"Content-Type": "application/xml", "consumerKey": consumer_key}
    payload = """<PreviewOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>MARKET</priceType>
                           <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice></limitPrice>
                           <Instrument>
                               <Product>
                                   <callPut>PUT</callPut>
                                   <expiryDay>{}</expiryDay>
                                   <expiryMonth>{}</expiryMonth>
                                   <expiryYear>2021</expiryYear>
                                   <securityType>OPTN</securityType>
                                   <strikePrice>{}</strikePrice>
                                   <symbol>SPY</symbol>
                               </Product>
                               <orderAction>SELL_CLOSE</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>1</quantity>
                           </Instrument>
                       </Order>
                   </PreviewOrderRequest>"""

    payload = payload.format(client_order_id,exp_day,exp_month,prev_strike_price)
    r = session.post(preview_order_url,header_auth=True,headers=headers,data=payload)
    if r.status_code == 200:
        r = json.loads(r.content)
        preview_id = r['PreviewOrderResponse']['PreviewIds'][0]['previewId']
        place_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/place.json'
        payload = """<PlaceOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <PreviewIds>
                          <previewId>{}</previewId>
                       </PreviewIds>
                       <Order>
                          <allOrNone>false</allOrNone>
                              <priceType>MARKET</priceType>
                              <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                              <marketSession>REGULAR</marketSession>
                              <stopPrice></stopPrice>
                              <limitPrice></limitPrice>
                          <Instrument>
                             <Product>
                                 <callPut>PUT</callPut>
                                 <expiryDay>{}</expiryDay>
                                 <expiryMonth>{}</expiryMonth>
                                 <expiryYear>2021</expiryYear>
                                 <securityType>OPTN</securityType>
                                 <strikePrice>{}</strikePrice>
                                 <symbol>SPY</symbol>
                             </Product>
                             <orderAction>SELL_CLOSE</orderAction>
                             <quantityType>QUANTITY</quantityType>
                             <quantity>1</quantity>
                          </Instrument>
                       </Order>
                    </PlaceOrderRequest>"""

        payload = payload.format(client_order_id,preview_id,exp_day,exp_month,prev_strike_price)
        r = session.post(place_order_url,header_auth=True,headers=headers,data=payload)
        if r.status_code == 200:
            print('SPY Short Closed!')
        else:
            print('SPY Close Place order failed - trying again')
            sell_close_order(prev_strike_price)
    else:
        print('SPY Close Preview order failed - trying again')
        sell_close_order(prev_strike_price)
        

rsi_below_40 = tf.keras.models.load_model('rsi_below_60_model/')
rsi_above_60 = tf.keras.models.load_model('rsi_above_60_model/')
macd_below_0 = tf.keras.models.load_model('macd_below_0_model/')
macd_above_0 = tf.keras.models.load_model('macd_above_0_model/')        
        
        
def stochastic(row):
    return ((row['price']-row['14_min'])/(row['14_max']-row['14_min']))*100


def gain(row):
    if row['change'] >= 0:
        return row['change']
    else:
        return 0

def loss(row):
    if row['change']<= 0:
        return row['change']
    else:
        return 0
    
def rsi(row):
    return 100 - (100 / (1+((row['prev_avg_gain']*13)+row['gain'])/-((row['prev_avg_loss']*13)+row['loss'])))
    
        
second = datetime.datetime.now().second

hour = datetime.datetime.now().hour
minute = datetime.datetime.now().minute


while hour < 8 :
    if second == 0:
        time.sleep(1)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute
    else:
        time.sleep(0.2)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute

print('between 8 and 8:30')

while hour > 7 and hour < 9 and minute < 30:
    if second == 0:
        time.sleep(1)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute
    else:
        time.sleep(0.2)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute
last_minute = minute - 1
print('8:30')

spy_data = pd.read_csv('spy_df.csv')
spy_data_dict['times'] = list(spy_data['time'])
spy_data_dict['prices'] = list(spy_data['price'])

long = False
short = False



while hour > 7 and hour < 15:
    

    if minute != last_minute and minute%5 == 0:
  
        last_minute = datetime.datetime.now().minute

        try:
            spy_price,curr_time = get_data()
        except:
            try:
                spy_price,curr_time = get_spy_data()
            except:
                pass

        strike_price = str(int(round(spy_price-1,0)))
        put_strike_price = str(int(round(spy_price+1,0)))
        spy_data_dict['times'].append(curr_time)
        spy_data_dict['prices'].append(spy_price)
        spy_df = pd.DataFrame({'time':spy_data_dict['times'],'price':spy_data_dict['prices']})
        spy_df.set_index('time',inplace=True)
        spy_df['change'] = spy_df['price'] - spy_df['price'].shift(1)
        spy_df['ema_26'] = spy_df['price'].ewm(span=26).mean()
        spy_df['ema_12'] = spy_df['price'].ewm(span=12).mean()
        spy_df['14_min'] = spy_df['price'].rolling(14).min()
        spy_df['14_max'] = spy_df['price'].rolling(14).max()
        spy_df['macd'] = spy_df['ema_12'] - spy_df['ema_26']
        spy_df['signal'] = spy_df['macd'].ewm(span=9).mean()
        spy_df['gain'] = spy_df.apply(gain, axis = 1)
        spy_df['loss'] = spy_df.apply(loss, axis = 1)
        spy_df['avg_gain'] = spy_df['gain'].rolling(14).mean()
        spy_df['avg_loss'] = spy_df['loss']. rolling(14).mean()
        spy_df['prev_avg_gain'] = spy_df['avg_gain'].shift(1)
        spy_df['prev_avg_loss'] = spy_df['avg_loss'].shift(1)
        spy_df['stochastic'] = spy_df.apply(stochastic, axis = 1)
        spy_df['rsi'] = spy_df.apply(rsi, axis = 1)

        spy_df.to_csv('spy_df.csv')

        if hour == 8 and minute == 30:
            time.sleep(1)
        else:

            fig, ax = plt.subplots()
            ax.plot(spy_df.reset_index().iloc[-20:].index, spy_df.iloc[-20:]['rsi'], color = 'blue', linewidth = 4)
            ax.plot(spy_df.reset_index().iloc[-20:].index, spy_df.iloc[-20:]['stochastic'], color = 'green', linewidth = 4)
            ax.axhline(y=70, color = 'black', linewidth = 4)
            ax.axhline(y=30, color = 'black', linewidth = 4)


            ax.axis('off')
            fig.savefig(f'spy_images/rsi.jpeg')
            plt.close(fig)

            fig, ax = plt.subplots()
            ax.plot(spy_df.reset_index().iloc[-20:].index, spy_df.iloc[-20:]['macd'], color = 'blue', linewidth = 4)
            ax.plot(spy_df.reset_index().iloc[-20:].index, spy_df.iloc[-20:]['signal'], color = 'green', linewidth = 4)
            ax.axhline(y=0, color = 'black', linewidth = 4)


            ax.axis('off')
            fig.savefig(f'spy_images/macd.jpeg')
            plt.close(fig)
            
            
            preds = ['none','none']
            img = keras.preprocessing.image.load_img(
                f"spy_images/rsi.jpeg", target_size=(img_height, img_width)
            )
            img_array = keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0) # Create a batch

            if spy_df.iloc[-1]['rsi'] < 50:
                predictions = rsi_below_40.predict(img_array)
                score = tf.nn.softmax(predictions[0])

                preds[0] = class_names[np.argmax(score)]

            elif spy_df.iloc[-1]['rsi'] > 50:
                predictions = rsi_above_60.predict(img_array)
                score = tf.nn.softmax(predictions[0])

                preds[0] = class_names[np.argmax(score)]

            img = keras.preprocessing.image.load_img(
                f"spy_images/macd.jpeg", target_size=(img_height, img_width)
            )
            img_array = keras.preprocessing.image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0) # Create a batch

            if spy_df.iloc[-1]['macd'] < 0:
                predictions = macd_below_0.predict(img_array)
                score = tf.nn.softmax(predictions[0])

                preds[1] = class_names[np.argmax(score)]

            elif spy_df.iloc[-1]['macd'] > 0:
                predictions = macd_above_0.predict(img_array)
                score = tf.nn.softmax(predictions[0])
                preds[1] = class_names[np.argmax(score)]
                
            if long and preds[0] == 'rsi_loss' and preds[1] == 'rsi_loss':
                sell_close_order(prev_strike_price)
                print(f'Close long SPY @ {spy_price}')
                long = False
                
            if short and preds[0] == 'rsi_gain' and preds[1] == 'rsi_gain':
                sell_close_order_put(prev_strike_price)
                print(f'Close short SPY @ {spy_price}')
                short = False
                

            if preds[0] == 'rsi_gain' and preds[1] == 'rsi_gain' and not long and not short:            
                buy_open_order()
                print(f'Long SPY @ {spy_price} {curr_time}')
                long = True
                long_enter = spy_price
                prev_strike_price = str(int(round(spy_price-1,0)))
                
            if preds[0] == 'rsi_loss' and preds[1] == 'rsi_loss' and not long and not short:
                buy_open_order_put()
                print(f'Short SPY @ {spy_price} {curr_time}')
                short = True
                short_enter = spy_price
                prev_strike_price = str(int(round(spy_price+1,0)))
            
            print(preds)
            time.sleep(1)
            second = datetime.datetime.now().second
            minute = datetime.datetime.now().minute
            hour = datetime.datetime.now().hour

            
    else:
        try:
            spy_price,curr_time = get_data()
        except:
            try:
                spy_price,curr_time = get_spy_data()
            except:
                pass
            

            
        if long and spy_price - long_enter <=-.40:
            sell_close_order(prev_strike_price)
            print(f'Stop loss long SPY @ {spy_price}')
            long = False
        
        if long and spy_price - long_enter >=.80:
            sell_close_order(prev_strike_price)
            print(f'Close long SPY for gain @ {spy_price}')
            long = False
        
        if short and spy_price - short_enter >=.40:
            sell_close_order_put(prev_strike_price)
            print(f'Stop loss short SPY @ {spy_price}')
            short = False
            
        if short and spy_price - short_enter <=-.80:
            sell_close_order_put(prev_strike_price)
            print(f'Close short SPY for gain @ {spy_price}')
            short = False
        
        
            
        time.sleep(1)
        second = datetime.datetime.now().second
        minute = datetime.datetime.now().minute
        hour = datetime.datetime.now().hour
            
if long:
    sell_close_order(prev_strike_price)
if short:
    sell_close_order_put(prev_strike_price)

Please accept agreement and enter text code from browser: 4A7VS
Enter Expiration Day5
Enter Expiration Month5
between 8 and 8:30
8:30


SPY Long Executed!
Long SPY @ 418.77 531000
['rsi_gain', 'rsi_gain']
['rsi_gain', 'rsi_gain']
['rsi_gain', 'rsi_loss']
SPY Long Closed!
Stop loss long SPY @ 418.3
['rsi_gain', 'rsi_loss']
SPY Long Executed!
Long SPY @ 418.46 5310201
['rsi_gain', 'rsi_gain']
['rsi_gain', 'rsi_loss']
['rsi_gain', 'rsi_loss']
SPY Long Closed!
Close long SPY @ 418.69
SPY Short Executed!
Short SPY @ 418.69 5310350
['rsi_loss', 'rsi_loss']
['rsi_loss', 'rsi_gain']
['rsi_loss', 'rsi_gain']
['rsi_loss', 'rsi_gain']
['rsi_loss', 'rsi_gain']
['rsi_loss', 'rsi_gain']
SPY Short Closed!
Close short SPY @ 418.75
SPY Long Executed!
Long SPY @ 418.75 531150
['rsi_gain', 'rsi_gain']
['rsi_gain', 'rsi_gain']
['rsi_gain', 'rsi_gain']
SPY Long Closed!
Close long SPY @ 418.47
SPY Short Place order failed
Short SPY @ 418.47 5311200
['rsi_loss', 'rsi_loss']


KeyboardInterrupt: 

In [3]:
testing = pd.read_csv('spy_df.csv')
spy_data_dict['times'] = list(testing['time'])
spy_data_dict['prices'] = list(testing['price'])

In [8]:
spy_df.reset_index().iloc[-20:]

,time,price,change,ema_26,ema_12,14_min,14_max,macd,signal,gain,loss,avg_gain,avg_loss,prev_avg_gain,prev_avg_loss,stochastic,rsi
9,4198500,416.65,0.06,416.222609,416.301103,NaN,NaN,0.078494,0.042858,0.06,0.00,NaN,NaN,NaN,NaN,NaN,NaN
10,4198550,416.63,-0.02,416.275448,416.361283,NaN,NaN,0.085835,0.052261,0.00,-0.02,NaN,NaN,NaN,NaN,NaN,NaN
11,419900,416.58,-0.05,416.312867,416.400170,NaN,NaN,0.087304,0.059787,0.00,-0.05,NaN,NaN,NaN,NaN,NaN,NaN
12,419950,415.76,-0.82,416.248099,416.289012,NaN,NaN,0.040914,0.055793,0.00,-0.82,NaN,NaN,NaN,NaN,NaN,NaN
13,4199100,415.37,-0.39,416.149478,416.132534,415.37,416.65,-0.016944,0.040576,0.00,-0.39,0.069286,-0.100714,NaN,NaN,0.000000,NaN
14,4199151,415.36,-0.01,416.064076,416.003121,415.36,416.65,-0.060954,0.019530,0.00,-0.01,0.069286,-0.101429,0.069286,-0.100714,0.000000,40.572716
15,4199200,415.64,0.28,416.019714,415.943113,415.36,416.65,-0.076601,-0.000253,0.28,0.00,0.086429,-0.101429,0.069286,-0.101429,21.705426,47.242069
16,4199250,415.95,0.31,416.012637,415.944238,415.36,416.65,-0.068399,-0.014196,0.31,0.00,0.108571,-0.100000,0.086429,-0.101429,45.736434,52.089281
17,4199301,415.81,-0.14,415.992617,415.922512,415.36,416.65,-0.070105,-0.025583,0.00,-0.14,0.108571,-0.110000,0.108571,-0.100000,34.883721,49.498998
18,4199351,415.95,0.14,415.988508,415.926925,415.36,416.65,-0.061583,-0.032888,0.14,0.00,0.118571,-0.110000,0.108571,-0.110000,45.736434,52.036416


In [9]:
preview_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/preview.json'
client_order_id = str(np.random.randint(1000000000, 2000000000))
headers = {"Content-Type": "application/xml", "consumerKey": consumer_key}
payload = """<PreviewOrderRequest>
                   <orderType>EQ</orderType>
                   <clientOrderId>{}</clientOrderId>
                   <Order>
                       <allOrNone>false</allOrNone>
                       <priceType>LIMIT</priceType>
                       <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                       <marketSession>REGULAR</marketSession>
                       <stopPrice></stopPrice>
                       <limitPrice>320</limitPrice>
                       <Instrument>
                           <Product>
                               <securityType>EQ</securityType>
                               <symbol>SPY</symbol>
                           </Product>
                           <orderAction>BUY</orderAction>
                           <quantityType>QUANTITY</quantityType>
                           <quantity>1</quantity>
                       </Instrument>
                   </Order>
               </PreviewOrderRequest>"""

payload = payload.format(client_order_id)
r = session.post(preview_order_url,header_auth=True,headers=headers,data=payload)

In [10]:
r.content

b'{"PreviewOrderResponse":{"orderType":"EQ","totalOrderValue":320,"previewTime":1620060006598,"dstFlag":true,"accountId":"50986220","optionLevelCd":2,"marginLevelCd":"MARGIN_TRADING_NOT_ALLOWED","cashBpDetails":{"settled":{"currentBp":11.38,"currentOor":0.00,"currentNetBp":11.38,"currentOrderImpact":320.00,"netBp":0.00},"settledUnsettled":{"currentBp":11.38}},"Order":[{"orderTerm":"GOOD_UNTIL_CANCEL","priceType":"LIMIT","limitPrice":320,"stopPrice":0,"marketSession":"REGULAR","allOrNone":false,"messages":{"Message":[{"description":"200|This order to purchase securities, if accepted and executed, will be paid for out of proceeds from the previous sale of securities in your account, which is to settle on or before the date by which payment for this purchase transaction is due.  If you subsequently enter an order to sell these securities before they are fully paid for, your account may be subject to a 90-day restriction under federal securities regulations.","code":2098,"type":"WARNING"}]

In [56]:
spy_data = pd.read_csv('spy_df.csv')
spy_data['next_five_change_max'] = float()
spy_data['next_five_change_min'] = float()
spy_data['next_ten_change_min'] = float()
spy_data['next_ten_change_max'] = float()
spy_data.reset_index(inplace=True)

In [57]:

def next_ten_max(row):
    i = int(row['index'])
    return spy_data.iloc[i:i+10]['price'].max()
spy_data['next_ten_max'] = spy_data.apply(next_ten_max, axis=1)


In [58]:
spy_data[:10]

,index,time,price,change,ema_26,ema_12,14_min,14_max,macd,signal,...,avg_loss,prev_avg_gain,prev_avg_loss,stochastic,rsi,next_five_change_max,next_five_change_min,next_ten_change_min,next_ten_change_max,next_ten_max
0,0,415184555,415.81,NaN,415.810000,415.810000,NaN,NaN,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,416.65
1,1,41518550,415.85,0.04,415.830769,415.831667,NaN,NaN,0.000897,0.000499,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,416.65
2,2,4151900,415.83,-0.02,415.830493,415.831016,NaN,NaN,0.000523,0.000509,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,416.65
3,3,41519557,415.83,0.00,415.830355,415.830695,NaN,NaN,0.000340,0.000452,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,416.65
4,4,41519101,415.83,0.00,415.830273,415.830506,NaN,NaN,0.000234,0.000387,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,416.65
5,5,4198301,416.25,0.42,415.914341,415.932465,NaN,NaN,0.018125,0.005195,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,416.65
6,6,4198350,416.50,0.25,416.018497,416.059109,NaN,NaN,0.040612,0.014158,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,416.65
7,7,4198400,416.39,-0.11,416.078356,416.128161,NaN,NaN,0.049805,0.022725,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,416.65
8,8,4198450,416.59,0.20,416.154192,416.219529,NaN,NaN,0.065336,0.032568,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,416.65
9,9,4198500,416.65,0.06,416.222609,416.301103,NaN,NaN,0.078494,0.042858,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,416.65


In [34]:
train = spy_data[:-50]

In [35]:
test = spy_data[-50:]

In [36]:
train.iloc[50]

time                    4.191420e+07
price                   4.144400e+02
change                 -1.700000e-01
ema_26                  4.149162e+02
ema_12                  4.148026e+02
14_min                  4.144400e+02
14_max                  4.150700e+02
macd                   -1.136234e-01
signal                 -1.163503e-01
gain                    0.000000e+00
loss                   -1.700000e-01
avg_gain                8.071429e-02
avg_loss               -8.428571e-02
prev_avg_gain           8.071429e-02
prev_avg_loss          -8.571429e-02
stochastic              0.000000e+00
rsi                     4.496480e+01
next_five_change_max    0.000000e+00
next_five_change_min    0.000000e+00
next_ten_change_min     0.000000e+00
next_ten_change_max     0.000000e+00
Name: 50, dtype: float64

In [ ]:
spy_dat

0